In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

import torch
from torch import nn
from torch.functional import F

print(torch.__version__)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

import wandb
wandb.login()


/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.12.1
Using mps device


wandb: Currently logged in as: mhrnciar. Use `wandb login --relogin` to force relogin


True

In [2]:
df = pd.read_csv('Data/cleaned.csv', index_col=0)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,102.5,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,32.0,169.5,30.1,0.349,47,1


In [3]:
X, y = df.drop('Outcome', axis=1).values, df.Outcome.values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_features=8, hidden1=20, hidden2=10, hidden3= 5, out_features=2):
        super().__init__()
        self.f_connected1 = nn.Linear(input_features, hidden1)
        self.f_connected2 = nn.Linear(hidden1, hidden2)
        self.f_connected3 = nn.Linear(hidden2, hidden3)
        self.out = nn.Linear(hidden3, out_features)

    def forward(self,x):
        x = F.relu(self.f_connected1(x))
        x = F.relu(self.f_connected2(x))
        x = F.relu(self.f_connected3(x))
        x = self.out(x)
        
        return x

model = NeuralNetwork()
print(model)

config = {'lr': 0.01, 'epochs': 500}

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])

run = wandb.init(project="basic-nn-torch", id="vivid-serenity-7", resume=True)
wandb.config.update(config)
wandb.watch(model)

wandb: WARNING Tried to auto resume run with id vivid-serenity-6 but id vivid-serenity-7 is set.


NeuralNetwork(
  (f_connected1): Linear(in_features=8, out_features=20, bias=True)
  (f_connected2): Linear(in_features=20, out_features=10, bias=True)
  (f_connected3): Linear(in_features=10, out_features=5, bias=True)
  (out): Linear(in_features=5, out_features=2, bias=True)
)


accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1_macro,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1_none/0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1_none/1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.69481
f1_macro,0.40996
f1_none/0,0.81992
f1_none/1,0.0
loss,0.65338


[]

In [7]:
final_losses = []

for i in range(config['epochs']):
    i += 1
    y_pred = model.forward(X_train)
    loss = loss_fn(y_pred, y_train)
    final_losses.append(loss)
    wandb.log({"loss": loss}, commit=False)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        y_pred = model(X_test)

        f1_none = f1_score(y_test, y_pred.argmax(1), average=None)
        f1_macro = f1_score(y_test, y_pred.argmax(1), average='macro')
        accuracy =  accuracy_score(y_test, y_pred.argmax(1))

        wandb.log({"f1_macro": f1_macro}, commit=False)

        f1_none = {"f1_none/" + str(e): v for e,v in enumerate(f1_none)}
        wandb.log(f1_none, commit=False)

        wandb.log({"accuracy": accuracy})

    if i % 10 == 0:
        print(f'Epoch {i}: {loss.item()}')

Epoch 10: 0.6079589128494263
Epoch 20: 0.5246925354003906
Epoch 30: 0.48756954073905945
Epoch 40: 0.46126314997673035
Epoch 50: 0.4435782730579376
Epoch 60: 0.42433246970176697
Epoch 70: 0.41680705547332764
Epoch 80: 0.4044867753982544
Epoch 90: 0.39609429240226746
Epoch 100: 0.3959476947784424
Epoch 110: 0.37971484661102295
Epoch 120: 0.3673239052295685
Epoch 130: 0.3928787410259247
Epoch 140: 0.35241156816482544
Epoch 150: 0.3522978127002716
Epoch 160: 0.3399188816547394
Epoch 170: 0.33551913499832153
Epoch 180: 0.32989588379859924
Epoch 190: 0.3270602226257324
Epoch 200: 0.37303903698921204
Epoch 210: 0.3306472897529602
Epoch 220: 0.3149607181549072
Epoch 230: 0.3154286742210388
Epoch 240: 0.31007200479507446
Epoch 250: 0.3040543794631958
Epoch 260: 0.3061225116252899
Epoch 270: 0.3095555007457733
Epoch 280: 0.3083738088607788
Epoch 290: 0.29541996121406555
Epoch 300: 0.29388800263404846
Epoch 310: 0.2889102101325989
Epoch 320: 0.28752097487449646
Epoch 330: 0.2845453917980194
Epoch

In [8]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_scatter(x=list(range(config['epochs'])), y=list(map(lambda x: x.item(), final_losses)))
fig.update_layout(xaxis_title='Epoch', yaxis_title='Loss')

In [9]:
torch.save(model.state_dict(), "models/model.pth")
wandb.save('runs/pima_run_2023-03-22')
wandb.finish()

accuracy,▁▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇████▇████▇█▇█████▇██▇██
f1_macro,▁▇▇██▇███▇▇▇█▇██████████████▇███████████
f1_none/0,▁▅▆▇▇▆▇▇▇▇▆▇▇▇▇█▆▇▇██▇████▇█▇█▇██▇▇██▇██
f1_none/1,▁▇▇██████▇▇█████████████████▇███████████
loss,█▆▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.9026
f1_macro,0.88951
f1_none/0,0.92754
f1_none/1,0.85149
loss,0.27921


In [10]:
predictions=[]

with torch.no_grad():
    for i, data in enumerate(X_test):
        y_pred=model(data)
        predictions.append(y_pred.argmax().item())

score = accuracy_score(y_test, predictions)
score

0.9025974025974026